In [1]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser
from os.path import join

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_visualization_subtitle,
    get_country_color_map,
)
from theme import apply_theme
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

In [2]:
data_release='2021-04-27'

df = pd.read_csv(join("..", "data", "Phase2.1SurvivalRSummariesPublic", "ToShare", "table.auc.toShare.csv"))
print(df.head())

# Rename columns
df = df.rename(columns={"calendar_month": "day"})

consistent_date = {
    '2020-Mar-Apr': "'20 Mar - '20 Apr",
    '2020-May-Jun': "'20 May - '20 Jun",
    '2020-Jul-Aug': "'20 Jul - '20 Aug",
    '2020-Sep-Oct': "'20 Sep - '20 Oct",
    '2020-Nov-2021-Jan': "'20 Nov - '21 Jan"
}

colors = ['#E79F00', '#0072B2', '#D45E00', '#CB7AA7', '#029F73', '#57B4E9']

# sites = ['META', 'BIDMC', 'FRBDX', 'NWU', 'MGB', 'UPENN', 'VA1', 'VA2', 'VA3', 'VA4', 'VA5']
sites = ['META', 'APHP', 'FRBDX', 'ICSM', 'BIDMC', 'MGB', 'UCLA', 'UMICH', 'UPENN', 'VA1', 'VA2', 'VA3', 'VA4', 'VA5']
site_colors = ['black', '#D45E00', '#0072B2', '#CB7AA7', '#E79F00', '#029F73', '#DBD03C', '#57B4E9', '#57B4E9', '#57B4E9', '#57B4E9', '#57B4E9']
sites = ['META', 'APHP', 'FRBDX', 'ICSM', 'UKFR', 'NWU', 'BIDMC', 'MGB', 'UCLA', 'UMICH', 'UPENN', 'UPITT', 'VA1', 'VA2', 'VA3', 'VA4', 'VA5']
site_colors = ['black', '#0072B2', '#0072B2', '#0072B2', '#0072B2', '#CB7AA7', '#D45E00', '#D45E00', '#D45E00', '#D45E00', '#D45E00', '#D45E00', '#D45E00', '#D45E00', '#D45E00','#D45E00','#D45E00']

df.siteid = df.siteid.apply(lambda x: x.upper())
df.day = df.day.apply(lambda x: consistent_date[x])
df.metrics = df.metrics.apply(lambda x: x.upper())

unique_days = df.day.unique().tolist()
metrics = df.metrics.unique().tolist()
print(df.metrics.unique().tolist())

df

   Unnamed: 0 metrics siteid     calendar_month       auc
0           1     cov   APHP       2020-Mar-Apr  0.761070
1           2     cov   APHP       2020-May-Jun  0.793907
2           3     cov   APHP       2020-Jul-Aug  0.789231
3           4     cov   APHP       2020-Sep-Oct  0.786162
4           5     cov   APHP  2020-Nov-2021-Jan  0.761940
['COV', 'DEM', 'LAB', 'CLS']


,Unnamed: 0,metrics,siteid,day,auc
0,1,COV,APHP,'20 Mar - '20 Apr,0.761070
1,2,COV,APHP,'20 May - '20 Jun,0.793907
2,3,COV,APHP,'20 Jul - '20 Aug,0.789231
3,4,COV,APHP,'20 Sep - '20 Oct,0.786162
4,5,COV,APHP,'20 Nov - '21 Jan,0.761940
...,...,...,...,...,...
355,356,CLS,META-FRANCE,'20 Mar - '20 Apr,0.536967
356,357,CLS,META-FRANCE,'20 May - '20 Jun,0.548439
357,358,CLS,META-FRANCE,'20 Jul - '20 Aug,0.550255
358,359,CLS,META-FRANCE,'20 Sep - '20 Oct,0.562879


# All Sites

In [3]:
point=alt.OverlayMarkDef(filled=False, fill='white', strokeWidth=2)

def plot_lab(df=None, metric='cov'):
    d = df.copy()
#     d = d[d.metrics == metric]
        
    plot = alt.Chart(
        d
    ).mark_line(
        point=point,
        size=3,
#         opacity=0.3
    ).encode(
        x=alt.X("day:N", title=None, axis=alt.Axis(labelAngle=-50, tickCount=10), scale=alt.Scale(padding=1), sort=unique_days),
        y=alt.Y("auc:Q", title=None, scale=alt.Scale(zero=True)),
        # color=alt.Color("siteid:N", scale=alt.Scale(domain=sites, range=site_colors)),
        color=alt.Color("metrics:N", scale=alt.Scale(domain=metrics, range=colors), title="Metric"),
        # shape=alt.Shape("metrics:N", scale=alt.Scale(domain=metrics, range=['circle', 'diamond', 'cross', 'triangle']))
    ).properties(
        width=120,
        height=200
    )

    plot = plot.facet(
        column=alt.Column("siteid:N", header=alt.Header(title=None), sort=sites)
    ).resolve_scale(color='shared')

    plot = plot.properties(
        title={
            "text": [
                f"AUC"
            ],
            "dx": 40,
#             "subtitle": [
#                 get_visualization_subtitle(data_release=data_release, with_num_sites=False)
#             ],
            "subtitleColor": "gray",
        }
    )

    return plot

plot = plot_lab(df=df)

# plot = alt.vconcat(*(
#     plot_lab(df=df, lab=lab) for lab in unique_sites
# ), spacing=30)

plot = apply_theme(
    plot,
    axis_y_title_font_size=16,
    title_anchor='start',
    header_label_orient='top',
    legend_orient='top',
    legend_title_orient='left',
    axis_label_font_size=14,
    header_label_font_size=16,
    point_size=100
)

plot

alt.FacetChart(...)

In [4]:
point=alt.OverlayMarkDef(filled=False, fill='white', strokeWidth=2)

def plot_lab(df=None, metric='cov'):
    d = df.copy()
    d = d[d.siteid=='META']
#     d = d[d.metrics == metric]
        
    plot = alt.Chart(
        d
    ).mark_line(
        point=point,
        size=3,
#         opacity=0.3
    ).encode(
        x=alt.X("day:N", title=None, axis=alt.Axis(labelAngle=-50, tickCount=10), scale=alt.Scale(padding=1), sort=unique_days),
        y=alt.Y("auc:Q", title=None, scale=alt.Scale(zero=True)),
        # color=alt.Color("siteid:N", scale=alt.Scale(domain=sites, range=site_colors)),
        color=alt.Color("metrics:N", scale=alt.Scale(domain=metrics, range=colors), title="Metric"),
        # shape=alt.Shape("metrics:N", scale=alt.Scale(domain=metrics, range=['circle', 'diamond', 'cross', 'triangle']))
    ).properties(
        width=300,
        height=250
    )

#     plot = plot.facet(
#         column=alt.Column("siteid:N", header=alt.Header(title=None), sort=sites)
#     ).resolve_scale(color='shared')

    plot = plot.properties(
        title={
            "text": [
                f"Meta-Analysis Of AUC"
            ],
            "dx": 30,
#             "subtitle": [
#                 get_visualization_subtitle(data_release=data_release, with_num_sites=False)
#             ],
            "subtitleColor": "gray",
        }
    )

    return plot

plot = plot_lab(df=df)

# plot = alt.vconcat(*(
#     plot_lab(df=df, lab=lab) for lab in unique_sites
# ), spacing=30)

plot = apply_theme(
    plot,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='bottom-right',
    legend_title_orient='top',
    axis_label_font_size=14,
    header_label_font_size=16,
    point_size=100
)

plot
save(plot,join("..", "result", "final-auc-meta.png"), scalefactor=8.0)



npm notice 
npm notice New minor version of npm available! 7.7.6 -> 7.18.1
npm notice Changelog: <https://github.com/npm/cli/releases/tag/v7.18.1>
npm notice Run `npm install -g npm@7.18.1` to update!
npm notice 


## Meta Country

In [5]:
point=alt.OverlayMarkDef(filled=False, fill='white', strokeWidth=2)

def plot_lab(df=None, metric='cov', country=None):
    d = df.copy()
    d = d[d.siteid==country]
#     d = d[d.metrics == metric]
        
    plot = alt.Chart(
        d
    ).mark_line(
        point=point,
        size=3,
#         opacity=0.3
    ).encode(
        x=alt.X("day:N", title=None, axis=alt.Axis(labelAngle=-50, tickCount=10), scale=alt.Scale(padding=1), sort=unique_days),
        y=alt.Y("auc:Q", title=None, scale=alt.Scale(zero=True)),
        # color=alt.Color("siteid:N", scale=alt.Scale(domain=sites, range=site_colors)),
        color=alt.Color("metrics:N", scale=alt.Scale(domain=metrics, range=colors), title="Metric"),
        # shape=alt.Shape("metrics:N", scale=alt.Scale(domain=metrics, range=['circle', 'diamond', 'cross', 'triangle']))
    ).properties(
        width=300,
        height=250
    )

#     plot = plot.facet(
#         column=alt.Column("siteid:N", header=alt.Header(title=None), sort=sites)
#     ).resolve_scale(color='shared')

    plot = plot.properties(
        title={
            "text": [
                country.replace("META-","")
            ],
            "dx": 30,
#             "subtitle": [
#                 get_visualization_subtitle(data_release=data_release, with_num_sites=False)
#             ],
            "subtitleColor": "gray",
        }
    )

    return plot

country_list=["META-USA", "META-FRANCE", "META-GERMANY"]
plot = alt.hconcat(*(
     plot_lab(df=df, metric="cov",country=country) for country in country_list
 ), spacing=30)

plot = apply_theme(
    plot,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='bottom-right',
    legend_title_orient='top',
    axis_label_font_size=14,
    header_label_font_size=16,
    point_size=100
)

plot
save(plot,join("..", "result", "final-auc-country.png"), scalefactor=8.0)




WARN Infinite extent for field "auc": [Infinity, -Infinity]
WARN Infinite extent for field "auc": [Infinity, -Infinity]


# Meta (Not used)

In [ ]:
def plot_lab(df=None, lab='Albumin (g/dL)'):
    d = df.copy()
    d = d[d.lab == lab]
    d = d[d.siteid == 'META']
        
    plot = alt.Chart(
        d
    ).mark_line(
        point=True,
        size=2,
#         opacity=0.3
    ).encode(
        x=alt.X("day:Q", title='Days Since Admission', axis=alt.Axis(tickCount=10), scale=alt.Scale(padding=1), sort=unique_days),
        y=alt.Y("value:Q", title='Fixed Effect', scale=alt.Scale(zero=False)),
#         color=alt.Color("siteid:N", scale=alt.Scale(domain=sites, range=site_colors), legend=None),
        color=alt.Color("period:N", scale=alt.Scale(domain=['early', 'late'], range=['#D45E00', '#0072B2'])),
        shape=alt.Shape("period:N", scale=alt.Scale(domain=['early', 'late'], range=['circle', 'diamond']), legend=None)
    ).properties(
        width=400,
        height=300
    )

#     plot = plot.facet(
#         column=alt.Column("siteid:N", title=None, header=alt.Header(title=None), sort=sites)
#     )

    plot = plot.properties(
        title={
            "text": [
                f"Pooled Lab Recovery Rate"
            ],
            "dx": 60,
            "subtitle": [
                lab,
                get_visualization_subtitle(data_release=data_release, with_num_sites=False)
            ],
            "subtitleColor": "gray",
        }
    )

    return plot

# plot = plot_lab(df=df)

plot = alt.hconcat(*(
    plot_lab(df=df, lab=lab) for lab in unique_sites
), spacing=30).resolve_scale(color='independent')

plot = apply_theme(
    plot,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='bottom',
    legend_title_orient='left',
    axis_label_font_size=14,
    header_label_font_size=16,
    point_size=100
)

plot    

# Below code is deprecated

In [ ]:
def risk(_d, metric='pos'):
    d = _d.copy()
    
    """
    DATA PREPROCESSING...
    """
    d.loc[d.site == 'combine', 'site'] = 'All Sites'
    d.cat = d.cat.apply(lambda x: {
        'L':'Low Risk', 
        'M': 'Medium Risk', 
        'H': 'High Risk',
        'H/M': 'High/Medium',
        'L/M': 'Low/Medium'
    }[x])
    
    """
    PLOT!
    """
    y_title = '% of Patients in Each Category' if metric == 'pos' else '% of Event in Each Category'
    colors = ['#7BADD1', '#427BB5', '#14366E'] if metric == 'pos' else ['#A8DED1', '#3FA86F', '#005A24'] if metric == 'ppv' else ['red', 'salmon']
    colorDomain = ['High/Medium', 'Low/Medium'] if metric == 'rr' else ['Low Risk', 'Medium Risk', 'High Risk']
    width = 300
    size = 50
    y_scale = alt.Scale(domain=[0, 1]) if metric == 'pos' or metric=='ppv' else alt.Scale()
    
    bar = alt.Chart(
        d
    ).transform_calculate(
        order="{'Low Risk':0, 'Medium Risk': 1, 'High Risk': 2}[datum.variable]"  
    ).transform_filter(
        {'field': 'metric', 'oneOf': [metric]}
    ).encode(
        x=alt.X("month:N", title='Month', scale=alt.Scale(domain=['Mar-Apr', 'May-Jun', 'Jul-Aug', 'Sep-Oct', 'Since Nov'])),
        y=alt.Y("value:Q", title=y_title, axis=alt.Axis(format='.0%'), scale=y_scale),
        color=alt.Color("cat:N", title='Category', scale=alt.Scale(domain=colorDomain, range=colors)),
        order="order:O"
    ).properties(
        width=width
    )
    
    if metric == 'pos':
        bar = bar.mark_bar(
            size=size, stroke='black'
        )
    else:
        bar = bar.mark_line(
            size=3, point=True, opacity=0.8
        )
    
    d['visibility'] = d['value'] > 0.08
    text = alt.Chart(
        d
    ).transform_filter(
        {'field': 'metric', 'oneOf': [metric]}
    ).mark_text(size=16, dx=0, dy=5, color='white', baseline='top', fontWeight=500).encode(
        x=alt.X('month:N'),
        y=alt.Y('value:Q', stack='zero'),
        detail='cat:N',
        text=alt.Text('value:Q', format='.0%'),
        order="order:O",
        opacity=alt.Opacity('visibility:N', scale=alt.Scale(domain=[True, False], range=[1, 0]))
    )
#     .transform_filter(
#         (f'datum.value > 0.10')
#     )
    
    if metric == 'pos':
        bar = (bar + text)
    
    bar = bar.facet(
        column=alt.Column('site:N', header=alt.Header(title=None)),
    )
    
    """
    COMBINE
    """
    res = bar.properties(
        title={
            "text": [
                f"Distribution of Risk Scores" if metric == 'pos' else f"Event Rate of Risk Scores"
            ],
            "dx": 80,
            "subtitle": [
                # lab, #.title(),
                get_visualization_subtitle(data_release=data_release, with_num_sites=False)
            ], 
            "subtitleColor": "gray",
        }
    )

    
    return res

In [ ]:
pos = risk(df, metric='pos')
ppv = risk(df, metric='ppv')

res = alt.vconcat(pos, ppv, spacing=30).resolve_scale(color='independent', x='independent')

res = apply_theme(
    res,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='right',
    header_label_font_size=16
)

res.display()

In [ ]:
d = df.copy()

width = 300
size = 50


"""
DATA PREPROCESSING...
"""
d.loc[d.site == 'combine', 'site'] = 'All Sites'
d.cat = d.cat.apply(lambda x: {
    'L':'Low', 
    'M': 'Medium', 
    'H': 'High',
    'H/M': 'H/M',
    'L/M': 'L/M'
}[x])
# d.cat = d.cat.apply(lambda x: {
#     'L':'Low Risk', 
#     'M': 'Medium Risk', 
#     'H': 'High Risk',
#     'H/M': 'High/Medium',
#     'L/M': 'Low/Medium'
# }[x])




"""
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%% TOP %%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
"""
metric='pos'

y_title = '% of Patients in Each Category'
y_scale = alt.Scale(domain=[0, 1])
colors = ['#7BADD1', '#427BB5', '#14366E']
# colorDomain = ['Low Risk', 'Medium Risk', 'High Risk']
colorDomain = ['Low', 'Medium', 'High']
# colorDomain = ['L', 'M', 'H']

bar = alt.Chart(
    d
).transform_calculate(
#     order="{'Low Risk':0, 'Medium Risk': 1, 'High Risk': 2}[datum.variable]"  
    order="{'L':0, 'M': 1, 'H': 2}[datum.variable]"  
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).encode(
    x=alt.X("month:N", title='Month', scale=alt.Scale(domain=['Mar-Apr', 'May-Jun', 'Jul-Aug', 'Sep-Oct', 'Since Nov'])),
    y=alt.Y("value:Q", title=y_title, axis=alt.Axis(format='.0%'), scale=y_scale),
    color=alt.Color("cat:N", title='Risk', scale=alt.Scale(domain=colorDomain, range=colors)),
    order="order:O"
).properties(
    width=width
)

bar = bar.mark_bar(
    size=size, stroke='black'
)


d['visibility'] = d['value'] > 0.08

text = alt.Chart(
    d
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).mark_text(size=16, dx=0, dy=5, color='white', baseline='top', fontWeight=500).encode(
    x=alt.X('month:N'),
    y=alt.Y('value:Q', stack='zero'),
    detail='cat:N',
    text=alt.Text('value:Q', format='.0%'),
    order="order:O",
    opacity=alt.Opacity('visibility:N', scale=alt.Scale(domain=[True, False], range=[1, 0]), legend=None)
)

if metric == 'pos':
    bar = (bar + text)

bar = bar.facet(
    column=alt.Column('site:N', header=alt.Header(title=None)),
    spacing=157
)

"""
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%% Bottom %%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
"""
metric='ppv'

y_title = '% of Event in Each Category'
colors = ['#A8DED1', '#3FA86F', '#005A24']
y_scale = alt.Scale(domain=[0, 1])

line = alt.Chart(
    d
).transform_calculate(
#     order="{'Low Risk':0, 'Medium Risk': 1, 'High Risk': 2}[datum.variable]"  
    order="{'L':0, 'M': 1, 'H': 2}[datum.variable]"  
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).encode(
    x=alt.X("month:N", title='Month', scale=alt.Scale(domain=['Mar-Apr', 'May-Jun', 'Jul-Aug', 'Sep-Oct', 'Since Nov'])),
    y=alt.Y("value:Q", title=y_title, axis=alt.Axis(format='.0%'), scale=y_scale),
    color=alt.Color("cat:N", title='Risk', scale=alt.Scale(domain=colorDomain, range=colors)),
    order="order:O"
).properties(
    width=width
)


line = line.mark_line(
    size=3, point=True, opacity=0.8
)


"""
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%% Bottom 2 %%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
"""
metric='rr'
y_title = 'Ratio Between Risk Categories'
colors = ['red', 'salmon']
# colorDomain = ['High/Medium', 'Low/Medium']
colorDomain = ['H/M', 'L/M']
y_scale = alt.Scale(domain=[0, 4.2])

line2 = alt.Chart(
    d
).transform_calculate(
#     order="{'High/Medium':0, 'Low/Medium': 1}[datum.variable]"  
    order="{'H/M':0, 'L/M': 1}[datum.variable]"  
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).encode(
    x=alt.X("month:N", title='Month', scale=alt.Scale(domain=['Mar-Apr', 'May-Jun', 'Jul-Aug', 'Sep-Oct', 'Since Nov'])),
    y=alt.Y("value:Q", title=y_title, axis=alt.Axis(format='.0%'), scale=y_scale),
    color=alt.Color("cat:N", title='Risk Ratio', scale=alt.Scale(domain=colorDomain, range=colors)),
    order="order:O"
).properties(
    width=width
)


line2 = line2.mark_line(
    size=3, point=True, opacity=0.8
)

# line = alt.layer(line, line2, data=d).resolve_scale(y='independent', color='independent')
# .facet(
#     column='site' # alt.Column('site:N', header=alt.Header(title=None))
# )

# for site in d.site.unique()

line = alt.concat(*(
    alt.layer(line, line2, title={
        "text": site,
        "dx": 200}).transform_filter(alt.datum.site == site).resolve_scale(y='independent', color='independent')
    for site in ['All Sites', 'MGB', 'BIDMC', 'VA1', 'VA2', 'VA3', 'VA4', 'VA5']
), spacing=20).resolve_scale(color='shared')

print(d.site.unique())

"""
COMBINE
"""
top = bar.properties(
    title={
        "text": [
            f"Distribution of Risk Scores"
        ],
        "dx": 80,
        "subtitle": [
            get_visualization_subtitle(data_release=data_release, with_num_sites=False)
        ], 
        "subtitleColor": "gray",
    }
)

bot = line.properties(
    title={
        "text": [
            f"Event Rate of Risk Scores"
        ],
        "dx": 80,
        "subtitle": [
            get_visualization_subtitle(data_release=data_release, with_num_sites=False)
        ], 
        "subtitleColor": "gray",
    }
)

res = alt.vconcat(top, bot, spacing=30).resolve_scale(color='independent')

"""
STYLE
"""

res = apply_theme(
    res,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='bottom',
    header_label_font_size=16
)

res

In [ ]:
d = df.copy()

width = 280
height = 200
height2 = 140
size = 50


"""
DATA PREPROCESSING...
"""
d.loc[d.site == 'combine', 'site'] = 'All Sites'
d.cat = d.cat.apply(lambda x: {
    'L':'Low', 
    'M': 'Medium', 
    'H': 'High',
    'H/M': 'H/M',
    'L/M': 'L/M'
}[x])
# d.cat = d.cat.apply(lambda x: {
#     'L':'Low Risk', 
#     'M': 'Medium Risk', 
#     'H': 'High Risk',
#     'H/M': 'High/Medium',
#     'L/M': 'Low/Medium'
# }[x])




"""
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%% TOP %%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
"""
metric='pos'

y_title = '% of Patients in Each Category'
y_scale = alt.Scale(domain=[0, 1])
colors = ['#7BADD1', '#427BB5', '#14366E']
# colorDomain = ['Low Risk', 'Medium Risk', 'High Risk']
colorDomain = ['Low', 'Medium', 'High']
# colorDomain = ['L', 'M', 'H']

bar = alt.Chart(
    d
).transform_calculate(
#     order="{'Low Risk':0, 'Medium Risk': 1, 'High Risk': 2}[datum.variable]"  
    order="{'L':0, 'M': 1, 'H': 2}[datum.variable]"  
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).encode(
    x=alt.X("month:N", title='Month', scale=alt.Scale(domain=['Mar-Apr', 'May-Jun', 'Jul-Aug', 'Sep-Oct', 'Since Nov']), axis=alt.Axis(grid=True)),
    y=alt.Y("value:Q", title=y_title, axis=alt.Axis(format='.0%'), scale=y_scale),
    color=alt.Color("cat:N", title='Risk', scale=alt.Scale(domain=colorDomain, range=colors)),
    order="order:O"
).properties(
    width=width,
    height=height
)

bar = bar.mark_bar(
    size=size, stroke='black'
)


d['visibility'] = d['value'] > 0.08

text = alt.Chart(
    d
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).mark_text(size=16, dx=0, dy=5, color='white', baseline='top', fontWeight=500).encode(
    x=alt.X('month:N'),
    y=alt.Y('value:Q', stack='zero'),
    detail='cat:N',
    text=alt.Text('value:Q', format='.0%'),
    order="order:O",
    opacity=alt.Opacity('visibility:N', scale=alt.Scale(domain=[True, False], range=[1, 0]), legend=None)
)

if metric == 'pos':
    bar = (bar + text)

bar = bar.facet(
    column=alt.Column('site:N', header=alt.Header(title=None)),
)

"""
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%% Bottom %%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
"""
metric='ppv'

y_title = '% of Risk Event'
colors = ['#A8DED1', '#3FA86F', '#005A24']
colors = ['#00A87E', '#00634B', 'black']
y_scale = alt.Scale(domain=[0, 0.6])

line = alt.Chart(
    d
).transform_calculate(
#     order="{'Low Risk':0, 'Medium Risk': 1, 'High Risk': 2}[datum.variable]"  
    order="{'L':0, 'M': 1, 'H': 2}[datum.variable]"  
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).encode(
    x=alt.X("month:N", title='Month', scale=alt.Scale(domain=['Mar-Apr', 'May-Jun', 'Jul-Aug', 'Sep-Oct', 'Since Nov']), axis=alt.Axis(grid=True, ticks=False, labels=False, domain=False, title=None)),
    y=alt.Y("value:Q", title=y_title, axis=alt.Axis(format='.0%'), scale=y_scale),
    color=alt.Color("cat:N", title='Risk', scale=alt.Scale(domain=colorDomain, range=colors)),
    order="order:O"
).properties(
    width=width,
    height=height2
)


line = line.mark_line(
    size=3, point=True, opacity=0.8
).facet(
    column=alt.Column('site:N', header=alt.Header(title=None)),
)



"""
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%% Bottom 2 %%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
"""
metric='rr'
y_title = 'Ratio Between Risks'
colors = ['#D45E00', '#351800']
# colorDomain = ['High/Medium', 'Low/Medium']
colorDomain = ['L/M', 'H/M']
y_scale = alt.Scale(domain=[0, 4.2])

line2 = alt.Chart(
    d
).transform_calculate(
#     order="{'High/Medium':0, 'Low/Medium': 1}[datum.variable]"  
    order="{'H/M':0, 'L/M': 1}[datum.variable]"  
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).encode(
    x=alt.X("month:N", title='Month', scale=alt.Scale(domain=['Mar-Apr', 'May-Jun', 'Jul-Aug', 'Sep-Oct', 'Since Nov']), axis=alt.Axis(grid=True)),
    y=alt.Y("value:Q", title=y_title, axis=alt.Axis(format='.0%'), scale=y_scale),
    color=alt.Color("cat:N", title='Risk Ratio', scale=alt.Scale(domain=colorDomain, range=colors)),
    order="order:O",
#     shape="site:N"
).properties(
    width=width,
    height=height2
)

line2 = line2.mark_line(
    size=3, opacity=0.8, point=True
).facet(
    column=alt.Column('site:N', header=alt.Header(title=None, labels=False)),
)






"""
COMBINE
"""
top = bar.properties(
    title={
        "text": [
            f"Distribution of Risk Scores"
        ],
        "dx": 180,
        "subtitle": [
            get_visualization_subtitle(data_release=data_release, with_num_sites=False)
        ], 
        "subtitleColor": "gray",
    }
)

line = line.properties(
    title={
        "text": [
            f"Event Rate of Risk Scores"
        ],
        "dx": 180,
        "subtitle": [
            get_visualization_subtitle(data_release=data_release, with_num_sites=False)
        ], 
        "subtitleColor": "gray",
    }
)

# line2 = line2.properties(
#     title={
#         "text": [
#             f"Risk Ratio"
#         ],
#         "dx": 180,
#         "subtitle": [
#             get_visualization_subtitle(data_release=data_release, with_num_sites=False)
#         ], 
#         "subtitleColor": "gray",
#     }
# )

res = alt.vconcat(top, line, line2, spacing=10).resolve_scale(color='independent')

"""
STYLE
"""

res = apply_theme(
    res,
    axis_y_title_font_size=14,
    axis_title_font_size=14,
    axis_label_font_size=12,
    title_anchor='start',
    legend_orient='left',
    header_label_font_size=16
)

res